处理方式与Baseline一样，来对比去除缺失率高的特征值的前后差异，baseline的总特征维度为271348维，最终的AUC值为0.745905，

In [39]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
import lightgbm as lgb
%matplotlib inline
import matplotlib.pyplot as plt

In [40]:
data = pd.read_csv('./datasets/train_data.csv')
target = pd.read_csv('./datasets/train_target.csv', names=['label'], header=None)

data.drop(data.columns[[0]], axis=1, inplace=True)  #删除某列函数
target = target.reset_index(drop=True)

In [41]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
           'adCategoryId', 'productId', 'productType']
vector_feature=['appIdAction','appIdInstall','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [42]:
X_train_encoded = data[['creativeSize']]
X_test_encoded = data[['creativeSize']]
feature_index = {}
feature_index['creativeSize'] = '0-0'
length = 1
oc_encoder = OneHotEncoder()
for feature in one_hot_feature:
    oc_encoder.fit(data[feature].reshape(-1, 1))
    train_a=oc_encoder.transform(data[feature].values.reshape(-1, 1))
    buf = train_a.toarray().shape[1]
    feature_index[feature] = str(length)+'-'+str(length+buf-1)
    length += buf
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [43]:
ct_encoder = CountVectorizer()
for feature in vector_feature:
    ct_encoder.fit(data[feature])
    train_a = ct_encoder.transform(data[feature])
    buf = len(ct_encoder.vocabulary_)
    feature_index[feature] = str(length)+'-' + str(length+int(buf)-1)
    length += buf
    X_train_encoded = sparse.hstack((X_train_encoded, train_a))

In [44]:
for nam, val in feature_index.items():
    print(nam, val)

creativeSize 0-0
LBS 1-636
age 637-642
carrier 643-646
consumptionAbility 647-649
education 650-657
gender 658-660
house 661-662
os 663-667
ct 668-732
marriageStatus 733-754
advertiserId 755-833
campaignId 834-971
creativeId 972-1144
adCategoryId 1145-1184
productId 1185-1217
productType 1218-1221
appIdAction 1222-6217
appIdInstall 6218-59233
interest1 59234-59346
interest2 59347-59417
interest3 59418-59418
interest4 59419-59419
interest5 59420-59546
kw1 59547-208532
kw2 208533-241312
kw3 241313-247613
topic1 247614-257594
topic2 257595-267391
topic3 267392-271347


In [46]:
y_train = np.array(target).squeeze()

In [47]:
clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=0.9,
        max_depth=-1, n_estimators=500, objective='binary',
        subsample=0.7, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.05, min_child_weight=60, random_state=2018, n_jobs=-1
    )
clf.fit(X_train_encoded, y_train, eval_set=[(X_train_encoded, y_train)], eval_metric='auc',early_stopping_rounds=50)

[1]	valid_0's auc: 0.616389
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.631905
[3]	valid_0's auc: 0.633141
[4]	valid_0's auc: 0.635386
[5]	valid_0's auc: 0.637963
[6]	valid_0's auc: 0.644473
[7]	valid_0's auc: 0.646393
[8]	valid_0's auc: 0.648793
[9]	valid_0's auc: 0.651887
[10]	valid_0's auc: 0.652665
[11]	valid_0's auc: 0.654435
[12]	valid_0's auc: 0.654632
[13]	valid_0's auc: 0.653847
[14]	valid_0's auc: 0.654361
[15]	valid_0's auc: 0.654984
[16]	valid_0's auc: 0.655116
[17]	valid_0's auc: 0.655326
[18]	valid_0's auc: 0.656419
[19]	valid_0's auc: 0.657634
[20]	valid_0's auc: 0.658018
[21]	valid_0's auc: 0.658036
[22]	valid_0's auc: 0.658218
[23]	valid_0's auc: 0.658645
[24]	valid_0's auc: 0.658619
[25]	valid_0's auc: 0.658467
[26]	valid_0's auc: 0.658431
[27]	valid_0's auc: 0.658398
[28]	valid_0's auc: 0.659545
[29]	valid_0's auc: 0.659922
[30]	valid_0's auc: 0.660636
[31]	valid_0's auc: 0.66066
[32]	valid_0's auc: 0.661394
[33]	valid_0's auc: 

[277]	valid_0's auc: 0.72857
[278]	valid_0's auc: 0.728649
[279]	valid_0's auc: 0.728828
[280]	valid_0's auc: 0.728899
[281]	valid_0's auc: 0.728979
[282]	valid_0's auc: 0.729192
[283]	valid_0's auc: 0.729261
[284]	valid_0's auc: 0.729343
[285]	valid_0's auc: 0.729404
[286]	valid_0's auc: 0.729507
[287]	valid_0's auc: 0.729614
[288]	valid_0's auc: 0.729713
[289]	valid_0's auc: 0.729802
[290]	valid_0's auc: 0.729866
[291]	valid_0's auc: 0.730007
[292]	valid_0's auc: 0.73007
[293]	valid_0's auc: 0.730137
[294]	valid_0's auc: 0.730253
[295]	valid_0's auc: 0.730367
[296]	valid_0's auc: 0.730435
[297]	valid_0's auc: 0.730535
[298]	valid_0's auc: 0.730618
[299]	valid_0's auc: 0.730744
[300]	valid_0's auc: 0.730818
[301]	valid_0's auc: 0.730885
[302]	valid_0's auc: 0.730979
[303]	valid_0's auc: 0.731085
[304]	valid_0's auc: 0.731276
[305]	valid_0's auc: 0.731356
[306]	valid_0's auc: 0.731484
[307]	valid_0's auc: 0.731607
[308]	valid_0's auc: 0.731726
[309]	valid_0's auc: 0.731779
[310]	valid_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=60, min_split_gain=0.0, n_estimators=500,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.9, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1)

In [48]:
fea_imp = clf.feature_importances_
top_index = np.argsort(fea_imp)
top_K = fea_imp[top_index[-50:]]
top_K

array([ 48,  48,  49,  49,  50,  50,  50,  51,  51,  51,  52,  59,  59,
        60,  60,  60,  61,  61,  61,  61,  62,  62,  64,  65,  67,  67,
        68,  69,  70,  70,  70,  71,  71,  72,  80,  81,  81,  83,  90,
        90,  93, 101, 108, 109, 131, 135, 139, 145, 157, 658])

In [113]:
def print_fea_name(index, fea_imp):
    for i in index:
        for name, val in feature_index.items():
            lis = val.split('-')
            num1 = int(lis[0])
            num2 = int(lis[1])
            if(num1 <= i & i <= num2):
                print(name, val, 'dimention:', i, 'importance:',fea_imp[i])
                break

In [114]:
index = top_index[-100:].tolist()
index.reverse()
print_fea_name(index,fea_imp)

creativeSize 0-0 dimention: 0 importance: 658
age 637-642 dimention: 639 importance: 157
interest2 59347-59417 dimention: 59390 importance: 145
productType 1218-1221 dimention: 1220 importance: 139
interest2 59347-59417 dimention: 59347 importance: 135
productType 1218-1221 dimention: 1221 importance: 131
adCategoryId 1145-1184 dimention: 1161 importance: 109
productType 1218-1221 dimention: 1218 importance: 108
ct 668-732 dimention: 668 importance: 101
interest2 59347-59417 dimention: 59406 importance: 93
interest1 59234-59346 dimention: 59296 importance: 90
interest2 59347-59417 dimention: 59359 importance: 90
interest2 59347-59417 dimention: 59399 importance: 83
advertiserId 755-833 dimention: 781 importance: 81
marriageStatus 733-754 dimention: 735 importance: 81
gender 658-660 dimention: 660 importance: 80
LBS 1-636 dimention: 164 importance: 72
interest2 59347-59417 dimention: 59367 importance: 71
age 637-642 dimention: 642 importance: 71
consumptionAbility 647-649 dimention: 649

Baseline中特征重要度排序为：
creativeSize age interest2 productType adCategoryId ct interest1 interest1 marriageStatus gender LBS consumptionAbility carrier kw2 interest5
并且特征kw1 apIdInstall appIdAction kw3 topic3的大部分importance为0 但是kw1、kw3、topic3有特例,考虑将min_df设为1000/879882

In [57]:
fea_imp.shape

(271348,)

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879882 entries, 0 to 879881
Data columns (total 32 columns):
aid                   879882 non-null int64
uid                   879882 non-null int64
advertiserId          879882 non-null int64
campaignId            879882 non-null int64
creativeId            879882 non-null int64
creativeSize          879882 non-null int64
adCategoryId          879882 non-null int64
productId             879882 non-null int64
productType           879882 non-null int64
LBS                   879882 non-null int64
age                   879882 non-null int64
appIdAction           879882 non-null object
appIdInstall          879882 non-null object
carrier               879882 non-null int64
consumptionAbility    879882 non-null int64
ct                    879882 non-null int64
education             879882 non-null int64
gender                879882 non-null int64
house                 879882 non-null int64
interest1             879882 non-null object
intere